In [1]:
import pandas as pd
import requests
import json



DATA_PATH="C:\\Projets\\WeatherData\\POC\\"

# load API keys from a file in folder .env, a line with the key name and the key value separated by a = sign
def load_api_keys():
    api_keys = {}
    with open(".env\\API_KEYS.cfg") as f:
        for line in f:
            key, val = line.split("=")
            key = key.strip()
            val = val.strip()            
            api_keys[key] = val
    return api_keys

api_keys = load_api_keys()



Free API for geocoding : 
https://openweathermap.org/api/geocoding-api  
https://overpass-api.de/

https://geocode.maps.co/ >> https://geocode.maps.co/search?street=555+5th+Ave&city=New+York&state=NY&postalcode=10017&country=US&api_key=api_key
1 req / sec max 100k / month

In [3]:
# geocoding with geocode.maps.co
city="BORDEAUX"
country="FR"

url_geo="https://geocode.maps.co/search?city="+city+"&country="+country+"&api_key="+api_keys["GEOCODE_MAPS_CO"]

response = requests.get(url_geo)
# get first element in response
geo_data = response.json()[0]
# get display_name, lat, lon and importance
display_name = geo_data["display_name"]
lat = geo_data["lat"]
lon = geo_data["lon"]
importance = geo_data["importance"]

print(f"{display_name} : lat={lat}, lon={lon}, importance={importance}")
if importance<0.5:
    print("Warning: importance is low, the location may be incorrect")


Bordeaux, Gironde, Nouvelle-Aquitaine, Metropolitan France, France : lat=44.841225, lon=-0.5800364, importance=0.8740050666982948


In [7]:
city="BORDEAUX"
country="FR"


# url_rech = "https://api.openaq.org/v2/locations?country="+country+"&city="+city+"&radius=20000&limit=100&page=1&offset=0&sort=asc&order_by=city"
url_rech = "https://api.openaq.org/v2/locations?country="+country+"&coordinates=44.83,-0.59&radius=20000&limit=100&page=1&offset=0&sort=asc&order_by=city"

headers = {"accept": "application/json"}

response = requests.get(url_rech, headers=headers)

print(response.text)

# Define the file path

json_file_path = DATA_PATH +city+ "_response.json"

# Write the response.text to the JSON file
with open(json_file_path, "w",encoding='utf-8') as file:
    # json.dump(response.text, file,ensure_ascii=False,indent=4)
    file.write(response.text)


{"meta":{"name":"openaq-api","license":"","website":"/","page":1,"limit":100,"found":12},"results":[{"id":3689,"city":"ATMO NOUVELLE-AQUITAINE","name":"NET-FR058A","entity":null,"country":"FR","sources":null,"isMobile":false,"isAnalysis":null,"parameters":[{"id":1,"unit":"µg/m³","count":37749,"average":16.14284461719223,"lastValue":7.9,"parameter":"pm10","displayName":"pm10 µg/m³","lastUpdated":"2024-03-01T19:00:00+00:00","parameterId":1,"firstUpdated":"2016-11-21T11:00:00+00:00","manufacturers":null},{"id":5,"unit":"µg/m³","count":42395,"average":14.325079226746874,"lastValue":6.3,"parameter":"no2","displayName":"no2 µg/m³","lastUpdated":"2024-03-01T19:00:00+00:00","parameterId":5,"firstUpdated":"2016-11-21T11:00:00+00:00","manufacturers":null},{"id":19843,"unit":"µg/m³","count":5993,"average":1.2898028552005427,"lastValue":0.5,"parameter":"no","displayName":"no µg/m³","lastUpdated":"2024-03-01T19:00:00+00:00","parameterId":19843,"firstUpdated":"2023-03-30T05:00:00+00:00","manufacture